In [ ]:
from pathlib import Path
from typing import Optional, Union

import simuran as smr
from simuran.loaders.allen_loader import AllenOphysLoader

from allensdk.brain_observatory.behavior.behavior_project_cache import (
    VisualBehaviorOphysProjectCache,
)
import pandas as pd
import dtale
import matplotlib.pyplot as plt
import numpy as np
from skm_pyutils.print import print_h5

In [ ]:
input_file_dir = Path(r"D:\AllenBrainObservatory\ophys_data")
MANIFEST_VERSION = "1.0.1"

In [ ]:
def get_path_to_nwb(session_id):
    fname = (
        input_file_dir
        / f"visual-behavior-ophys-{MANIFEST_VERSION}"
        / "behavior_ophys_experiments"
        / f"behavior_ophys_experiment_{session_id}.nwb"
    )
    return fname

In [ ]:
def setup_table(input_file_dir: Union[str, Path]) -> pd.DataFrame:
    cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=input_file_dir)
    experiments = cache.get_ophys_experiment_table()
    # dtale.show(experiments).open_browser()
    return experiments

def filter_table(table: pd.DataFrame, inplace: bool = True) -> pd.DataFrame:
    """
    _summary_

    Parameters
    ----------
    table : pd.DataFrame
        _description_
    inplace : bool, optional
        _description_, by default True

    Returns
    -------
    pd.DataFrame
        _description_
    """
    genotype = "Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6f)/wt"
    # Alternatively
    values = {
        "project_code": ["VisualBehaviorMultiscope"],
        "full_genotype": [genotype],
    }
    filters = []
    for k, v in values.items():
        filters.append(table[k].isin(v))
    full_mask = np.logical_and.reduce(np.array(filters))
    filtered_table = table[full_mask]
    return filtered_table


In [ ]:
def establish_analysis(rec):
    # Temp fn here
    def print_info(recording, *args, **kwargs):
        print(recording)
        print(*args, **kwargs)
        return vars(recording)

    ah = smr.AnalysisHandler()
    ah.add_fn(print_info, rec)

    return ah

In [ ]:
cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=input_file_dir)

# Should support params in multiple formats of metadata
table = setup_table(input_file_dir)

# Step 2 - Read a filtered table, can explore with d-tale etc. before continuing (JASP)
filtered_table = filter_table(table)
# rc = RecordingContainer.from_table(filtered_table, "allen", AllenOphysLoader)
# TODO I think this step is awkward

# TODO TEMP let us check a single Allen first

## This could be a data class to make it simpler
for idx, row in filtered_table.iterrows():
    row_as_dict = row.to_dict()
    row_as_dict[filtered_table.index.name] = idx
    break

recording = smr.Recording()
recording.loader = AllenOphysLoader(cache=cache)
# loader2 = loader_from_str("allen_ophys")(cache=cache)

# TODO This should support different types, file, dict, series, etc
recording.set_metadata(row_as_dict)

# TODO this should come with params / loader setting
recording.available = ["signals"]
print(recording)

## TODO is there a setup step? for recording to set paths?

# This will call load in the background
# If not already loaded
# recording.get_blah()
# Alternatively can call
recording.new_load()

In [ ]:
dataset = recording.signals

In [ ]:
## Now let us do things with the table
print(dataset.__dict__)
print(dataset)

In [ ]:
dataset.cell_specimen_table

In [ ]:
path_to_nwb = get_path_to_nwb(idx)
path_to_nwb

In [ ]:
print_h5(path_to_nwb)

In [ ]:
type(dataset)

In [ ]:
print(dataset.to_nwb())

In [ ]:
dff_traces = dataset.dff_traces
print(dff_traces)
ophys_timestamps = dataset.ophys_timestamps
print(ophys_timestamps)
print(dataset.metadata['ophys_frame_rate'])
print(np.mean(np.diff(ophys_timestamps)))
print(1 / 11.0)

In [ ]:
dataset.list_data_attributes_and_methods()